In [14]:
import pandas as pd

cand_h = ["DATA_GERACAO_V","HORA_GERACAO_V","ANO_ELEICAO","NUM_TURNO","DESCRICAO_ELEICAO","SIGLA_UF","SIGLA_UE","CODIGO_MUNICIPIO","NOME_MUNICIPIO","NUM_ZONA","NUM_SECAO","CODIGO_CARGO","DESCRICAO_CARGO","NUM_VOTAVEL","QTDE_VOTOS","DATA_GERACAO_C",
"HORA_GERACAO_c",
"ANO_ELEICAO_c",
"NUM_TURNO",
"SIGLA_UF",
"SIGLA_UE",
"DESCRICAO_UE",
"CODIGO_CARGO",
"DESCRICAO_CARGO",
"NOME_CANDIDATO",
"SEQUENCIAL_CANDIDATO",
"NUMERO_CANDIDATO",
"CPF_CANDIDATO",
"NOME_URNA_CANDIDATO",
"COD_SITUACAO_CANDIDATURA",
"DES_SITUACAO_CANDIDATURA",
"NUMERO_PARTIDO",
"SIGLA_PARTIDO",
"NOME_PARTIDO",
"CODIGO_LEGENDA",
"SIGLA_LEGENDA",
"COMPOSICAO_LEGENDA",
"NOME_LEGENDA",
"CODIGO_OCUPACAO",
"DESCRICAO_OCUPACAO",
"DATA_NASCIMENTO",
"NUM_TITULO_ELEITORAL_CANDIDATO",
"IDADE_DATA_ELEICAO",
"CODIGO_SEXO",
"DESCRICAO_SEXO",
"COD_GRAU_INSTRUCAO",
"DESCRICAO_GRAU_INSTRUCAO",
"CODIGO_ESTADO_CIVIL",
"DESCRICAO_ESTADO_CIVIL",
"CODIGO_COR_RACA",
"DESCRICAO_COR_RACA",
"CODIGO_NACIONALIDADE",
"DESCRICAO_NACIONALIDADE",
"SIGLA_UF_NASCIMENTO",
"CODIGO_MUNICIPIO_NASCIMENTO",
"NOME_MUNICIPIO_NASCIMENTO",
"DESPESA_MAX_CAMPANHA",
"COD_SIT_TOT_TURNO",
"DESC_SIT_TOT_TURNO",
"NM_EMAIL"]

candidato = pd.read_csv("data/votacao_candidato_2016.csv",delimiter="\t")
candidato.head()

,2016,1,RO,00019,11,PREFEITO,12,7370,26/04/2017,19:29:07,...,PARDA,1.3,BRASILEIRA NATA,RO.2,-3,GUAJARÁ-MIRIM.1,-1,1.4,ELEITO,PDT-GUAJARAMIRIM@HOTMAIL.COM
2016,1,RO,19,11,PREFEITO,15,9722,26/04/2017,19:29:07,2016,...,BRANCA,1,BRASILEIRA NATA,RO,-3,GUAJARÁ-MIRIM,0,4,NÃO ELEITO,ABENTO66@HOTMAIL.COM
2016,1,RO,19,11,PREFEITO,31,2756,26/04/2017,19:29:07,2016,...,PRETA,1,BRASILEIRA NATA,AC,-3,PLÁCIDO DE CASTRO,-1,4,NÃO ELEITO,GERSON.MAIA@HOTMAIL.COM
2016,1,RO,35,11,PREFEITO,13,26629,26/04/2017,19:29:07,2016,...,BRANCA,1,BRASILEIRA NATA,SP,-3,SÃO PAULO,-1,4,NÃO ELEITO,ROBERTO-SOBRINHO@UOL.COM.BR
2016,1,RO,35,11,PREFEITO,14,55656,26/04/2017,19:29:07,2016,...,BRANCA,1,BRASILEIRA NATA,PR,-3,FOZ DO IGUAÇU,0,6,2º TURNO,WDADVOCACIA@HOTMAIL.COM
2016,1,RO,35,11,PREFEITO,15,33379,26/04/2017,19:29:07,2016,...,1,BRASILEIRA NATA,RO,-3,PORTO VELHO,NÃO ELEITO,WILLIAMESPIMENTEL@HOTMAIL.COM,NaN,NaN,NaN


# Sqls da estruturaçao do Votacao seção de 2016 com as agregações regionais:



## Agregação Macro
```mysql

create view votacao_secao_2016_macro as 
    select CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,sum(cast(QTDE_VOTOS as int)) as total_votos 
    from (
    select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
    from votacao_secao_2016 as v 
    join aux_municipio as m 
    on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
    group by CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO;
```


## Agregação UF
```mysql
create view votacao_secao_2016_uf as 
    select CODIGO_MACRO,NOME_MACRO,UF,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,sum(cast(QTDE_VOTOS as int)) as total_votos 
   from (
       select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
       from votacao_secao_2016 as v 
       join aux_municipio as m 
       on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
   group by CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,UF;
```

## Agregação Meso
```mysql
    create view votacao_secao_2016_meso as 
    select CODIGO_MACRO, NOME_MACRO, CODIGO_MESO, NOME_MESO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, sum(cast(QTDE_VOTOS as int)) as total_votos 
    from (
        select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
        from votacao_secao_2016 as v 
        join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
    group by CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,CODIGO_MESO,NOME_MESO;
```

## Agregação Micro
```mysql

    create view votacao_secao_2016_micro as 
    select CODIGO_MACRO, NOME_MACRO, CODIGO_MESO, NOME_MESO, CODIGO_MICRO, NOME_MICRO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, sum(cast(QTDE_VOTOS as int)) as total_votos 
    from (
        select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
        from votacao_secao_2016 as v 
        join aux_municipio as m 
        on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
    group by CODIGO_MACRO, NOME_MACRO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, CODIGO_MESO, NOME_MESO, CODIGO_MICRO, NOME_MICRO;
```

## Junção com tabela candidatos

## Agregação Macro
```mysql

create view votacao_candidato_2016_macro as
    select * from votacao_secao_2016_macro as v
    join candidato_2016 as c 
    on  v.SIGLA_UE=c.SIGLA_UE AND 
        v.CODIGO_CARGO=c.CODIGO_CARGO AND 
        v.NUM_VOTAVEL=c.NUMERO_CANDIDATO AND 
        v.NUM_TURNO=c.NUM_TURNO

```


## Agregação UF
```mysql
create view votacao_secao_2016_uf as 
    select CODIGO_MACRO,NOME_MACRO,UF,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,sum(cast(QTDE_VOTOS as int)) as total_votos 
   from (
       select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
       from votacao_secao_2016 as v 
       join aux_municipio as m 
       on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
   group by CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,UF;
```

## Agregação Meso
```mysql
    create view votacao_secao_2016_meso as 
    select CODIGO_MACRO, NOME_MACRO, CODIGO_MESO, NOME_MESO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, sum(cast(QTDE_VOTOS as int)) as total_votos 
    from (
        select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
        from votacao_secao_2016 as v 
        join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
    group by CODIGO_MACRO,NOME_MACRO,SIGLA_UE,NUM_VOTAVEL,CODIGO_CARGO,NUM_TURNO,CODIGO_MESO,NOME_MESO;
```

## Agregação Micro
```mysql

    create view votacao_secao_2016_micro as 
    select CODIGO_MACRO, NOME_MACRO, CODIGO_MESO, NOME_MESO, CODIGO_MICRO, NOME_MICRO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, sum(cast(QTDE_VOTOS as int)) as total_votos 
    from (
        select m.*,v.NUM_VOTAVEL,v.CODIGO_CARGO,v.NUM_TURNO,v.QTDE_VOTOS,v.SIGLA_UE 
        from votacao_secao_2016 as v 
        join aux_municipio as m 
        on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO) as o 
    group by CODIGO_MACRO, NOME_MACRO, SIGLA_UE, NUM_VOTAVEL, CODIGO_CARGO, NUM_TURNO, CODIGO_MESO, NOME_MESO, CODIGO_MICRO, NOME_MICRO;
```